In [1]:
import yaml
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json
import pandas as pd
import glob

def load_and_normalize_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    df = pd.json_normalize(data)
    return df

def add_algorithm_prefix(df):
    algorithm_name = df['algorithm'].iloc[0]
    metric_columns = [col for col in df.columns if col.startswith('metrics.')]
    new_column_names = {col: f"{algorithm_name}_{col.split('.')[-1]}" for col in metric_columns}
    df.rename(columns=new_column_names, inplace=True)
    return df

def get_combined_df(dataset, path):
    json_files = glob.glob(f'{path}/{dataset}/*.json')
    dataframes = [load_and_normalize_json(file) for file in json_files]
    dataframes = [add_algorithm_prefix(df) for df in dataframes]
    combined_df = dataframes[0]
    for df in dataframes[1:]:
        if dataset == '05-puzzles':
            combined_df = pd.merge(combined_df, df, on=['env_grid_search.num_agents', 'env_grid_search.seed', 'env_grid_search.map_name'], suffixes=('', '_dup'))
        elif dataset == '03-warehouse':
            combined_df = pd.merge(combined_df, df, on=['env_grid_search.num_agents', 'env_grid_search.seed'], suffixes=('', '_dup'))
        elif dataset in ['01-random', '02-mazes', '04-movingai']:
            combined_df = pd.merge(combined_df, df, on=['env_grid_search.num_agents', 'env_grid_search.map_name'], suffixes=('', '_dup'))
        elif dataset in ['06-pathfinding']:
            combined_df = pd.merge(combined_df, df, on=['env_grid_search.seed', 'env_grid_search.map_name'], suffixes=('', '_dup'))

    # Drop duplicate columns resulting from the merge
    combined_df = combined_df.loc[:, ~combined_df.columns.str.endswith('_dup')]
    if 'algorithm' in combined_df.columns:
        combined_df.drop(columns=['algorithm'], inplace=True)
    return combined_df


def add_congestion(data_dict, algos, combined_df, path_to_maps):
    with open(f'{path_to_maps}/maps.yaml', 'r') as f:
        maps = yaml.safe_load(f)
    traversable_cells = {}
    for m in maps:
        cells = 0
        for i in maps[m]:
            if i in ['.', '@', '&', '$', '!']:
                cells += 1
        traversable_cells[m] = cells
    num_agents = combined_df['env_grid_search.num_agents'].max()
    filtered_df = combined_df[combined_df['env_grid_search.num_agents'] == combined_df['env_grid_search.num_agents'].max()]
    for algo in algos:
        density = []
        if len(traversable_cells) == 1:
            for index, row in filtered_df.iterrows():
                density.append((float(num_agents)/cells)/row[f'{algo}_avg_agents_density'])
        else:
            for index, row in filtered_df.iterrows():
                density.append((float(num_agents)/traversable_cells[row['env_grid_search.map_name']])/row[f'{algo}_avg_agents_density'])
        data_dict[algo]['Congestion'] = np.array(density).mean()


In [ ]:
random_list_2 = [84]
random_list_1 = [24,32,40]
#random_list_3 = [ 32, 64, 96, 128, 160, 192 ]
[232,256 ]
random_list_4 = [ 64, 128, 192,256]
random_list = random_list_1  
raw_Data_path = "/mnt/c/Users/Oguzhan Karaarslan/Desktop/repos/pogema-lacam-dcc/raw_data_MAPF"
map_type = '01-random'
benchmark_flag = 0
for i in random_list: 
    if benchmark_flag == 1:
        df = load_and_normalize_json(f"{raw_Data_path}/{map_type}/LaCAM.json")
    else:
        df = load_and_normalize_json(f"/mnt/c/Users/Oguzhan Karaarslan/Desktop/repos/pogema-lacam-dcc/algorithms/experiments/{map_type}/LaCAM.json")
    
    data_dict = {'LaCAM':{'Congestion':None}}
    df = add_algorithm_prefix(df)
    
    if benchmark_flag == 1:
        comb_df = get_combined_df(f"{map_type}",f"{raw_Data_path}")
    else:
        comb_df = get_combined_df(f"{map_type}","/mnt/c/Users/Oguzhan Karaarslan/Desktop/repos/pogema-lacam-dcc/algorithms/experiments")
    df_8 = comb_df[comb_df['env_grid_search.num_agents'] == i]

    if benchmark_flag == 1:
        add_congestion(data_dict, ['LaCAM'], df_8, f"{raw_Data_path}/{map_type}/")
    else:
        add_congestion(data_dict, ['LaCAM'], df_8, f"/mnt/c/Users/Oguzhan Karaarslan/Desktop/repos/pogema-lacam-dcc/algorithms/experiments/{map_type}/")

    print(data_dict['LaCAM']['Congestion'])

1.0200341285697183
1.0492705948333791
1.066230461330829


In [71]:
random_list_2 = [84]
random_list_1 = [8,16,24]
#random_list_3 = [ 32, 64, 96, 128, 160, 192 ]
[232,256 ]
random_list_4 = [ 64, 128, 192,256]
random_list = random_list_1  
raw_Data_path = "/mnt/c/Users/Oguzhan Karaarslan/Desktop/repos/pogema-lacam-dcc/raw_data_MAPF"
map_type = '01-random'
benchmark_flag = 0
for i in random_list: 
    if benchmark_flag == 1:
        df = load_and_normalize_json(f"{raw_Data_path}/{map_type}/LaCAM.json")
    else:
        df = load_and_normalize_json(f"/mnt/c/Users/Oguzhan Karaarslan/Desktop/repos/pogema-lacam-dcc/algorithms/experiments/{map_type}/LaCAM.json")
    
    data_dict = {'LaCAM':{'Congestion':None}}
    df = add_algorithm_prefix(df)
    
    if benchmark_flag == 1:
        comb_df = get_combined_df(f"{map_type}",f"{raw_Data_path}")
    else:
        comb_df = get_combined_df(f"{map_type}","/mnt/c/Users/Oguzhan Karaarslan/Desktop/repos/pogema-lacam-dcc/algorithms/experiments")
    df_8 = comb_df[comb_df['env_grid_search.num_agents'] == i]

    if benchmark_flag == 1:
        add_congestion(data_dict, ['LaCAM'], df_8, f"{raw_Data_path}/{map_type}/")
    else:
        add_congestion(data_dict, ['LaCAM'], df_8, f"/mnt/c/Users/Oguzhan Karaarslan/Desktop/repos/pogema-lacam-dcc/algorithms/experiments/{map_type}/")
    #
    print(data_dict['LaCAM']['Congestion']) # benchmark : 0.8319653741907043
    # benchmark one map: 0.8183799272303593

0.8446329766370461
0.9480064696651915
1.0191500881197106


In [ ]:
##{'LaCAM': {'Congestion': 0.8327010122025179}} - 
##{'LaCAM': {'Congestion': 0.9476686737645119}} - 

In [ ]:
###########    BENCHMARK   ########### 
### 8 ----- {'LaCAM': {'Congestion': 0.8313665077986148}}
### 16 ----- {'LaCAM': {'Congestion': 0.9470853146846667}}